# Manage a Server on Azure

This notebook can be used to launch a server using Azure CLI.  It is designed to be run via the included Docker container, but it can be run locally if the appropriate tools are installed.  

This script assumes you have already installed the az-server package and logged in using the `az login` command.  

To install the package use:
`carme package install az-server`

To log into the Azure CLI:
`az login`


#Remove the comment below.     

In [ ]:
#!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ECJUCE9R9 to authenticate.


### Do you want to run the commands or just see them?  
Below we have set the carme option dryrun so that commands are printed and not executed. 

In [11]:
#To run for real, just set dryrun=''
#dryrun= ''
dryrun='--dryrun'

### Print Available Commands
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [17]:
!cat ../../commands/az-server.yaml


login: "az login"
login_sa: "echo 'Service account login not yet available for Azure'"
create_resource_group: "az group create --name {resource_group} --location {location}"
create_server: "az vm create -n {server_name} -g {resource_group} --image {image_name} --authentication-type {server_auth} --admin-username User --admin-password Pass"
create_storage:  "az storage account create --resource-group={resource_group} --location={location} --sku=Standard_LRS --name={storage_account} --kind=Storage"
get_storage_key: "az storage account keys list --account-name={storage_account} --resource-group={resource_group} --output=json | jq .[0].value -r"
create_keyvault: "az keyvault create --name={cluster_name} --resource-group={resource_group} --location={location} --enabled-for-template-deployment true"
backup_private_key: "az keyvault secret set --vault-name={cluster_name} --name=id-rsa-{cluster_name} --file=~/.ssh/id_rsa_{cluster_name}"
backup_public_key: "az keyvault secret set --vault-name={

### Create Resource Groups 
Google calls them projects.  Azure calles them resource groups. Either way you need one. This useful to track spending and also ensure you delete all resources at the end. 


In [14]:
!carme cmd az-server create_resource_group $dryrun

carme: [INFO] Running the command: create_resource_group
carme: [INFO] Template: az group create --name {resource_group} --location {location}
carme: [INFO] Values: az group create --name carme --location eastus2


### Review the Server Configuration
This will create your Kubernetes Cluster. You have to wait for about 5 minutes before this finishes creating.


In [15]:
!cat ../../config/az-server.yaml

azure_image: carmelabs/azurecli
email: 'jason@analyticsdojo.com'    #Email Associated with the account.
#Start of Azure Configuration
resource_group: carme     #Resource groups are used to separate projects.
server_name: carme_server
server_image: UbuntuLTS         #Bionic 18.04 (LTS)Artful 17.10 Xenial 16.04 (LTS)
server_admin: carme_user
server_auth: '--generate-ssh-keys'
storage_account: twitter          #For use with Pachyderm
container_name: twitter           #For use with Pachyderm
storage_name: pach2-disk.vhd   #For use with Pachyderm
storage_size: 1                #For use with Pachyderm Storage account size in GB.
location: eastus2                 #Selection from `az account list-locations`.


In [ ]:
#Create the Server 

In [20]:
!carme cmd az-server create_server $dryrun

carme: [INFO] Running the command: create_server
carme: [INFO] Template: az vm create -n {server_name} -g {resource_group} --image {server_image} --size {server_size} --authentication-type {server_auth}
carme: [INFO] Values: az vm create -n carme_server -g carme --image UbuntuLTS --size Standard_DS2_v2 --authentication-type --generate-ssh-keys


### WAIT FOR A WHILE
This can take a few minutes. 



In [ ]:
#gcloud container clusters get-credentials carme
!az vm open-port --port 80 --resource-group myResourceGroup --name myVM

In [ ]:
#Check to see if we have Kubectl working. 
!kubectl cluster-info


In [ ]:
#Check notes with Kubectl
!kubectl get node


### Helm Installation.  
We are going to be utilizing Helm for  installations of a variety of analytics tools.  This command will install Tiller on your cluster.  As they say, "Happy Helming!" 

A critical factor for Helm is that you have the same version running locally and via your machine.  If you run helm version and you have the right version, then you should be fine.

```
Client: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.6.2", GitCommit:"be3ae4ea91b2960be98c07e8f73754e67e87963c", GitTreeState:"clean"}
```

To install the appropriate version: 

```
curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get > get_helm.sh
chmod 700 get_helm.sh
RUN get_helm.sh --version v2.6.2

```


In [ ]:
#This may need to be run more than once if you get a "cannot connect to server."
!helm version

In [ ]:
# Both ACS and GKE install tiller on the server.
!helm init --client-only

In [ ]:
#Run this command if the Client > Server version
!helm init --upgrade

In [ ]:
#Verify Versions are equal
!helm version

### Enabling Autoscaling (optional)

This should launch a pod within your kubernetes cluster that will handle autoscaling of the cluster. Note that this seems to take a while and may even timeout. Consider opening and running in a terminal session. 

In [ ]:
#Google only. Currently Not possible with azure
!carme cluster autoscale

#### That is it! You now have your own Kubernetes cluster that is ready to go. 

### Resize a Cluster


In [ ]:
#Scale the cluster 
!carme cluster class_size

In [ ]:
#Stop the cluster, effectively setting the size to 0.
!carme cluster stop

In [ ]:
#Set the cluster to the normal size.
!carme cluster normal_size

### Deleting a Kubernetes Cluster

This will delete the Kubernetes cluster by deleting the entire project. This will prefent any future charges. 

In [ ]:
#Always delete the namespace first. 
!carme cluster delete